In [1]:
from desdeo.emo.options.algorithms import nsga3_options, emo_constructor
from desdeo.problem.testproblems.dtlz2_problem import dtlz2
from desdeo.tools.scalarization import add_group_DF_funcs

/home/rishi/projects/DESDEO/desdeo/__init__.py:9: UserWarning: 
            
The following required highly recommended executables cannot be found: bonmin, cbc, ipopt

            DESDEO relies on powerful 3rd party solvers to solve multiobjective
            optimization problems.  Without these solvers, sub-optimal defaults
            will be used instead, which can lead to not optimal, or even wrong,
            results.

            It is highly recommended to have these solvers available
            in the environment DESDEO is utilized!

            For more information, see DESDEO's documentation: https://desdeo.readthedocs.io/en/latest/howtoguides/installing/#third-party-optimizers
            
  _check_executables(required_executables)


In [2]:
problem = dtlz2(n_objectives=3, n_variables=10)

In [3]:
solutions = {
    "dm1": {"f_1": 0.95, "f_2": 0.2, "f_3": 0.15},
    "dm2": {"f_1": 0.21, "f_2": 0.95, "f_3": 0.21},
    "dm3": {"f_1": 0.23, "f_2": 0.24, "f_3": 0.98},
}

aspirations = {
    "dm1": {"f_1": 0.9, "f_2": 0.18, "f_3": 0.1},
    "dm2": {"f_1": 0.15, "f_2": 0.9, "f_3": 0.15},
    "dm3": {"f_1": 0.2, "f_2": 0.2, "f_3": 0.8},
}

reservations = {
    "dm1": {"f_1": 0.98, "f_2": 0.25, "f_3": 0.2},
    "dm2": {"f_1": 0.3, "f_2": 0.99, "f_3": 0.25},
    "dm3": {"f_1": 0.3, "f_2": 0.3, "f_3": 0.99},
}

In [4]:
problem_, symbols = add_group_DF_funcs(problem, aspirations, reservations)

In [5]:
solver, extras = emo_constructor(nsga3_options, problem=problem_)


In [6]:
results = solver()

In [7]:
import plotly.express as ex

In [8]:
fig = ex.scatter_3d(
    x=extras.archive.results.optimal_outputs["f_1"],
    y=extras.archive.results.optimal_outputs["f_2"],
    z=extras.archive.results.optimal_outputs["f_3"],
    opacity=0.7,
    size=[5]*len(extras.archive.results.optimal_outputs["f_1"]),
    size_max=8,
)
# marker line width = 0
fig.update_traces(marker=dict(line=dict(width=0)))

fig.add_scatter3d(
    x=[solutions[dm]["f_1"] for dm in solutions],
    y=[solutions[dm]["f_2"] for dm in solutions],
    z=[solutions[dm]["f_3"] for dm in solutions],
    mode="markers+text",
    marker=dict(size=5, color="red"),
    text=["dm1"],
    textposition="top center",
)

In [9]:
fig = ex.scatter_3d(
    x=results.optimal_outputs["f_1"],
    y=results.optimal_outputs["f_2"],
    z=results.optimal_outputs["f_3"],
)

fig.add_scatter3d(
    x=[solutions[dm]["f_1"] for dm in solutions],
    y=[solutions[dm]["f_2"] for dm in solutions],
    z=[solutions[dm]["f_3"] for dm in solutions],
    mode="markers+text",
    marker=dict(size=5, color="red"),
    text=["dm1", "dm2", "dm3"],
    textposition="top center",
)

# ranges
fig.update_layout(
    scene=dict(
        xaxis=dict(range=[0, 1]),
        yaxis=dict(range=[0, 1]),
        zaxis=dict(range=[0, 1]),
    )
)

In [10]:
solver, extras = emo_constructor(nsga3_options, problem=problem)

solver()

EMOResult(optimal_variables=shape: (91, 10)
┌──────────┬──────────┬──────────┬──────────┬───┬──────────┬──────────┬──────────┬──────────┐
│ x_1      ┆ x_2      ┆ x_3      ┆ x_4      ┆ … ┆ x_7      ┆ x_8      ┆ x_9      ┆ x_10     │
│ ---      ┆ ---      ┆ ---      ┆ ---      ┆   ┆ ---      ┆ ---      ┆ ---      ┆ ---      │
│ f64      ┆ f64      ┆ f64      ┆ f64      ┆   ┆ f64      ┆ f64      ┆ f64      ┆ f64      │
╞══════════╪══════════╪══════════╪══════════╪═══╪══════════╪══════════╪══════════╪══════════╡
│ 0.758846 ┆ 0.19998  ┆ 0.495665 ┆ 0.498382 ┆ … ┆ 0.505137 ┆ 0.471918 ┆ 0.531565 ┆ 0.504906 │
│ 0.248956 ┆ 0.822837 ┆ 0.499755 ┆ 0.498421 ┆ … ┆ 0.499487 ┆ 0.503409 ┆ 0.4891   ┆ 0.501795 │
│ 0.595172 ┆ 0.294079 ┆ 0.509014 ┆ 0.497703 ┆ … ┆ 0.488265 ┆ 0.512062 ┆ 0.506971 ┆ 0.504348 │
│ 0.436374 ┆ 0.101936 ┆ 0.498592 ┆ 0.498476 ┆ … ┆ 0.504324 ┆ 0.511177 ┆ 0.517642 ┆ 0.483552 │
│ 0.785552 ┆ 0.498246 ┆ 0.49845  ┆ 0.497158 ┆ … ┆ 0.504598 ┆ 0.50681  ┆ 0.504289 ┆ 0.476253 │
│ …        ┆ …  

In [11]:
from desdeo.problem import MathParser

front = extras.archive.results.optimal_outputs

expressions = [(func.symbol, MathParser().parse(func.func)) for func in problem_.scalarization_funcs]

res = front.select(*[expr.alias(symbol) for symbol, expr in expressions])
res

df_dm1,df_dm2,df_dm3
f64,f64,f64
-0.000007,-0.000021,-0.000032
-0.002308,-0.003286,-0.000011
-0.000191,-0.001369,-0.000013
-0.000006,-0.00321,-0.001927
-0.000008,-0.000021,-0.000041
…,…,…
-0.000007,-0.000032,-0.000039
-0.002169,-0.000012,-0.003303
-0.001103,-0.133169,-0.001368


In [12]:
fig = ex.scatter_3d(
    x=res["df_dm1"],
    y=res["df_dm2"],
    z=res["df_dm3"],
    color=front["f_1"] * front["f_2"] * front["f_3"],
    opacity=0.7,
    size=[5]*len(res["df_dm1"]),
    size_max=8,
)

fig.update_traces(marker=dict(line=dict(width=0)))



In [13]:
from desdeo.tools.non_dominated_sorting import non_dominated

is_nd = non_dominated(res["df_dm1", "df_dm2", "df_dm3"].to_numpy())

fig = ex.scatter_3d(
    x=res["df_dm1"],
    y=res["df_dm2"],
    z=res["df_dm3"],
    color=is_nd,
    opacity=0.7,
    size=[5]*len(res["df_dm1"]),
    size_max=8,
)

fig.update_traces(marker=dict(line=dict(width=0)))


In [14]:
fig = ex.scatter_3d(
    x=front["f_1"],
    y=front["f_2"],
    z=front["f_3"],
    color=front["f_1"] * front["f_2"] * front["f_3"],
    opacity=0.7,
    size=[5]*len(extras.archive.results.optimal_outputs["f_1"]),
    size_max=8,
)
# marker line width = 0
fig.update_traces(marker=dict(line=dict(width=0)))
# add solutions

fig.add_scatter3d(
    x=[solutions[dm]["f_1"] for dm in solutions],
    y=[solutions[dm]["f_2"] for dm in solutions],
    z=[solutions[dm]["f_3"] for dm in solutions],
    mode="markers+text",
    marker=dict(size=5, color="red"),
    text=["dm1", "dm2", "dm3"],
    textposition="top center",
)

In [15]:
fig = ex.scatter_3d(
    x=front["f_1"],
    y=front["f_2"],
    z=front["f_3"],
    color=is_nd,
    opacity=0.7,
    size=[5]*len(extras.archive.results.optimal_outputs["f_1"]),
    size_max=8,
)
# marker line width = 0
fig.update_traces(marker=dict(line=dict(width=0)))
# add solutions

fig.add_scatter3d(
    x=[solutions[dm]["f_1"] for dm in solutions],
    y=[solutions[dm]["f_2"] for dm in solutions],
    z=[solutions[dm]["f_3"] for dm in solutions],
    mode="markers+text",
    marker=dict(size=5, color="red"),
    text=["dm1", "dm2", "dm3"],
    textposition="top center",
)

In [16]:
# Fairness tests

from desdeo.tools.scalarization import add_iopis_fairness

In [17]:
problem.get_ideal_point()

{'f_1': 0.0, 'f_2': 0.0, 'f_3': 0.0}

In [18]:
problem_, symbols = add_iopis_fairness(problem, solutions, ideal=problem.get_ideal_point())

In [19]:
solver, extras = emo_constructor(nsga3_options, problem=problem_)
results = solver()

In [26]:
fig = ex.scatter_3d(
    x=results.optimal_outputs["f_1"],
    y=results.optimal_outputs["f_2"],
    z=results.optimal_outputs["f_3"],
)

fig.add_scatter3d(
    x=[solutions[dm]["f_1"] for dm in solutions],
    y=[solutions[dm]["f_2"] for dm in solutions],
    z=[solutions[dm]["f_3"] for dm in solutions],
    mode="markers+text",
    marker=dict(size=5, color="red"),
    text=["dm1", "dm2", "dm3"],
    textposition="top center",
)

# ranges
fig.update_layout(
    scene=dict(
        xaxis=dict(range=[0, 1]),
        yaxis=dict(range=[0, 1]),
        zaxis=dict(range=[0, 1]),
    )
)

In [20]:
fig = ex.scatter_3d(
    x=extras.archive.results.optimal_outputs["f_1"],
    y=extras.archive.results.optimal_outputs["f_2"],
    z=extras.archive.results.optimal_outputs["f_3"],
    opacity=0.7,
    size=[5]*len(extras.archive.results.optimal_outputs["f_1"]),
    size_max=8,
)
# marker line width = 0
fig.update_traces(marker=dict(line=dict(width=0)))

fig.add_scatter3d(
    x=[solutions[dm]["f_1"] for dm in solutions],
    y=[solutions[dm]["f_2"] for dm in solutions],
    z=[solutions[dm]["f_3"] for dm in solutions],
    mode="markers+text",
    marker=dict(size=5, color="red"),
    text=["dm1", "dm2", "dm3"],
    textposition="top center",
)

In [21]:
expressions = [(func.symbol, MathParser().parse(func.func)) for func in problem_.scalarization_funcs]

res = front.select(*[expr.alias(symbol) for symbol, expr in expressions])

fig = ex.scatter_3d(
    x=res["min"],
    y=res["sum"],
    z=res["ratio"],
    color=front["f_1"] * front["f_2"] * front["f_3"],
    opacity=0.7,
    size=[5]*len(res["min"]),
    size_max=8,
)

fig.update_traces(marker=dict(line=dict(width=0)))

In [22]:
is_nd = non_dominated(res["min", "sum", "ratio"].to_numpy())

fig = ex.scatter_3d(
    x=res["min"],
    y=res["sum"],
    z=res["ratio"],
    color=is_nd,
    opacity=0.7,
    size=[5]*len(res["min"]),
    size_max=8,
)

fig.update_traces(marker=dict(line=dict(width=0)))

In [25]:
fig = ex.scatter_3d(
    x=front["f_1"],
    y=front["f_2"],
    z=front["f_3"],
    color=is_nd,
    opacity=0.7,
    size=[5]*len(front["f_1"]),
    size_max=8,
)
# marker line width = 0
fig.update_traces(marker=dict(line=dict(width=0)))
# add solutions

fig.add_scatter3d(
    x=[solutions[dm]["f_1"] for dm in solutions],
    y=[solutions[dm]["f_2"] for dm in solutions],
    z=[solutions[dm]["f_3"] for dm in solutions],
    mode="markers+text",
    marker=dict(size=5, color="red"),
    text=["dm1", "dm2", "dm3"],
    textposition="top center",
)

In [24]:
res["min"].min()

0.9730113766151424